In [28]:
import pandas as pd
import numpy as np
from scipy.special import softmax

usecols=['Respondent',        #id
  'FormalEducation',    #degree
  'UndergradMajor',     #major
  'LanguageWorkedWith', 'DatabaseWorkedWith', 'PlatformWorkedWith', 'IDE', 'OperatingSystem', #skills
  'YearsCoding',        #projectExperience
  'Hobby',              #hobby
  'DevType',            #desiredJob
  ]

user_path = r'E:\OneDrive - National University of Singapore\code\NUS-ISS\Project1\IRS-PM-2024-09-10-GRP8-JobRecommendationSystem\Recommendation_System\New_User\user_clean.csv'

df = pd.read_csv(user_path, usecols=usecols)
df = df.rename(columns={
  'Respondent': 'id',
  'FormalEducation': 'degree',
  'UndergradMajor': 'major',
  'LanguageWorkedWith': 'skills_1',
  'DatabaseWorkedWith': 'skills_2',
  'PlatformWorkedWith': 'skills_3',
  'IDE': 'skills_4',
  'OperatingSystem': 'skills_5',
  'YearsCoding': 'projectExperience',
  'Hobby': 'hobby',
  'DevType': 'desiredJob',
})

df.head()

,id,hobby,degree,major,desiredJob,projectExperience,skills_1,skills_2,skills_3,skills_4,skills_5
0,96,Yes,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Back-end developer;Database administrator;Fron...,6-8 years,C#;Python,SQL Server,Windows Desktop or Server,Visual Studio;Visual Studio Code,Windows
1,105,Yes,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Another engineering discipline (ex. civil, ele...",Back-end developer;Full-stack developer;QA or ...,12-14 years,Perl;R;SQL,NaN,Linux,RStudio;Vim,Linux-based
2,122,Yes,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A business discipline (ex. accounting, finance...",Back-end developer;Embedded applications or de...,0-2 years,JavaScript;Ruby;SQL;HTML;CSS,MongoDB;Redis;SQL Server,Heroku,Sublime Text,MacOS
3,294,Yes,Some college/university study without earning ...,NaN,Back-end developer;Front-end developer;Full-st...,3-5 years,Java;JavaScript;PHP;Scala;SQL;Swift;HTML;CSS,MySQL;Elasticsearch,Android;Firebase,Android Studio;IntelliJ;PHPStorm;Sublime Text,MacOS
4,297,Yes,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A business discipline (ex. accounting, finance...",Back-end developer,0-2 years,NaN,NaN,NaN,NaN,NaN


In [29]:
df['skills_1'] = df['skills_1'].str.split(';')
df['skills_2'] = df['skills_2'].str.split(';')
df['skills_3'] = df['skills_3'].str.split(';')
df['skills_4'] = df['skills_4'].str.split(';')
df['skills_5'] = df['skills_5'].str.split(';')

df['skills'] = df[['skills_1', 'skills_2', 'skills_3', 'skills_4', 'skills_5']].apply(lambda x: sum([i for i in x if isinstance(i, list)], []), axis=1)
df.drop(columns=['skills_1', 'skills_2', 'skills_3', 'skills_4', 'skills_5'], inplace=True)
# df = pd.concat([df_onehot, df['skills']], axis=1)

In [30]:
df['hobby'] = df['hobby'].apply(lambda x: 'True' if x == 'Yes' else 'False')
df['desiredJob'] = df['desiredJob'].str.split(';')
df['desiredJob'] = df[['desiredJob']].apply(lambda x: sum([i for i in x if isinstance(i, list)], []), axis=1)
# df['desiredJob'] = df['desiredJob'].apply(lambda x: [] if pd.isnull(str(x)) else x)

In [31]:
new_data = {
    'id': 0,
    'hobby': 'True',
    'degree': 'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
    'major': 'Computer science, computer engineering, or software engineering',
    'desiredJob': [],
    'projectExperience': '0-2 years',
    'skills': ['visual studio', 'python', 'c', 'pycharm', 'matlab', 'C++', 'SQL', 'Eclipse']
}

# new_data = {
#     'id': 0,
#     'hobby': 'True',
#     'degree': 'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
#     'major': 'Computer science, computer engineering, or software engineering',
#     'desiredJob': [],
#     'projectExperience': '0-2 years',
#     'skills':  ['python', 'javascript', 'c++', 'c']
# }

# new_data = {
#     'id': 0,
#     'hobby': 'True',
#     'degree': 'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
#     'major': 'Computer science, computer engineering, or software engineering',
#     'desiredJob': [],
#     'projectExperience': '0-2 years',
#     'skills':  ['python', 'javascript', 'c++', 'c', 'java', 'html']
# }

df = df._append(new_data, ignore_index=True)

In [32]:
df_onehot = pd.get_dummies(df['degree'])
df['degree'] = df_onehot.apply(lambda x: x.values.tolist(), axis=1)
df_onehot = pd.get_dummies(df['major'])
df['major'] = df_onehot.apply(lambda x: x.values.tolist(), axis=1)
df_onehot = pd.get_dummies(df['projectExperience'])
df['projectExperience'] = df_onehot.apply(lambda x: x.values.tolist(), axis=1)

In [33]:
df

,id,hobby,degree,major,desiredJob,projectExperience,skills
0,96,True,"[False, True, False, False, False, False, Fals...","[False, False, False, False, False, False, Tru...","[Back-end developer, Database administrator, F...","[False, False, False, False, False, False, Fal...","[C#, Python, SQL Server, Windows Desktop or Se..."
1,105,True,"[False, False, False, True, False, False, Fals...","[False, False, False, False, False, True, Fals...","[Back-end developer, Full-stack developer, QA ...","[False, True, False, False, False, False, Fals...","[Perl, R, SQL, Linux, RStudio, Vim, Linux-based]"
2,122,True,"[False, True, False, False, False, False, Fals...","[True, False, False, False, False, False, Fals...","[Back-end developer, Embedded applications or ...","[True, False, False, False, False, False, Fals...","[JavaScript, Ruby, SQL, HTML, CSS, MongoDB, Re..."
3,294,True,"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[Back-end developer, Front-end developer, Full...","[False, False, False, False, False, False, Fal...","[Java, JavaScript, PHP, Scala, SQL, Swift, HTM..."
4,297,True,"[False, True, False, False, False, False, Fals...","[True, False, False, False, False, False, Fals...",[Back-end developer],"[True, False, False, False, False, False, Fals...",[]
...,...,...,...,...,...,...,...
1382,65734,False,"[False, True, False, False, False, False, Fals...","[False, False, False, False, False, False, Tru...","[Back-end developer, Mobile developer]","[False, False, False, False, False, False, Fal...",[]
1383,70764,True,"[True, False, False, False, False, False, Fals...","[False, False, False, False, False, False, Tru...",[Full-stack developer],"[False, True, False, False, False, False, Fals...",[]
1384,90067,True,"[False, True, False, False, False, False, Fals...","[False, False, False, False, False, False, Tru...","[Full-stack developer, Mobile developer]","[False, False, False, False, False, False, Fal...",[]
1385,97126,False,"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",[Full-stack developer],"[True, False, False, False, False, False, Fals...",[]


In [34]:
# Function to calculate Jaccard similarity for binary vectors (one-hot encoded)
def jaccard_similarity_binary(vector1, vector2):
    set1, set2 = set(np.where(vector1 == 1)[0]), set(np.where(vector2 == 1)[0])
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if union == 0:
        return 0
    return intersection / union

# Function to calculate Jaccard similarity for lists of strings (case insensitive)
def jaccard_similarity(list1, list2):
    set1, set2 = set(map(str.lower, list1)), set(map(str.lower, list2))
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if union == 0:
        return 0
    return intersection / union


# Convert string representations of lists back to actual lists
list_columns = ['degree', 'major', 'desiredJob', 'projectExperience', 'skills']
for col in list_columns:
    df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Separate the new user (last row) from the rest of the data
new_user = df.iloc[-1]
other_users = df[:-1]

# Similarity calculation for each feature

# 1. Similarity for 'hobby' (boolean comparison)
hobby_similarity = other_users['hobby'].apply(lambda x: 100 if x == new_user['hobby'] else 0)

# 2. Similarity for one-hot encoded features (degree, major, projectExperience)
onehot_similarity = []
for idx, other_user in other_users.iterrows():
    degree_sim = jaccard_similarity_binary(np.array(new_user['degree']), np.array(other_user['degree']))
    major_sim = jaccard_similarity_binary(np.array(new_user['major']), np.array(other_user['major']))
    project_sim = jaccard_similarity_binary(np.array(new_user['projectExperience']), np.array(other_user['projectExperience']))
    onehot_similarity.append((degree_sim + major_sim + project_sim) / 0.03)

# 3. Similarity for lists of strings (desiredJob, skills)
list_similarity = []
for idx, other_user in other_users.iterrows():
    desiredJob_sim = jaccard_similarity(new_user['desiredJob'], other_user['desiredJob'])
    skills_sim = jaccard_similarity(new_user['skills'], other_user['skills'])
    list_similarity.append((desiredJob_sim + skills_sim) / 0.02)

# Combine similarities with equal weights
total_similarities = hobby_similarity * 0.05 + np.array(onehot_similarity) * 0.35 + np.array(list_similarity) * 0.6

# Create a DataFrame to display similarities
similarities_df = pd.DataFrame({
    'UserIndex': other_users.id,
    'HobbySimilarity': hobby_similarity,
    'OneHotSimilarity': onehot_similarity,
    'ListSimilarity': list_similarity,
    'TotalSimilarity': total_similarities
})

# Sort by total similarity
# similarities_df = similarities_df.sort_values(by='UserIndex', ascending=True)

# Display the top results
# Apply softmax to the 'TotalSimilarity' column
similarities_df['TotalSimilarity_softmax'] = softmax(similarities_df['TotalSimilarity'])

# Display the top results
similarities_df


,UserIndex,HobbySimilarity,OneHotSimilarity,ListSimilarity,TotalSimilarity,TotalSimilarity_softmax
0,96,100,33.333333,7.692308,21.282051,3.555785e-12
1,105,100,33.333333,3.571429,18.809524,3.000063e-13
2,122,100,33.333333,2.777778,18.333333,1.863475e-13
3,294,100,0.000000,2.083333,6.250000,1.053413e-18
4,297,100,33.333333,0.000000,16.666667,3.519649e-14
...,...,...,...,...,...,...
1381,62517,100,33.333333,0.000000,16.666667,3.519649e-14
1382,65734,0,33.333333,0.000000,11.666667,2.371521e-16
1383,70764,100,33.333333,0.000000,16.666667,3.519649e-14
1384,90067,100,33.333333,0.000000,16.666667,3.519649e-14


In [35]:
data = np.load(r'E:\OneDrive - National University of Singapore\code\NUS-ISS\Project1\IRS-PM-2024-09-10-GRP8-JobRecommendationSystem\Recommendation_System\KGAT\trained_model\KGAT\ours\pretrain0\cf_scores.npy')
# Replace -inf values with 0
data[data == -np.inf] = np.min(data[data != -np.inf])

# Normalize each row
data = data / np.linalg.norm(data, axis=1, keepdims=True)
print(data)

[[-0.00243886 -0.01366318 -0.00229861 ... -0.01367053 -0.01351818
  -0.0014719 ]
 [-0.00187401 -0.01343531 -0.00178946 ... -0.01343441 -0.01333417
  -0.00075654]
 [-0.00154783 -0.01351052  0.00191604 ... -0.01353971 -0.01336512
   0.00274765]
 ...
 [-0.00159803 -0.01344985  0.00241874 ... -0.01349136 -0.01326161
   0.00316403]
 [-0.00153399 -0.01336901  0.00260701 ... -0.01340353 -0.01320143
   0.00324562]
 [-0.00106834 -0.01357744  0.00207702 ... -0.01364959 -0.01337631
   0.00268559]]


In [36]:
# Ensure that similarities_df['TotalSimilarity_softmax'] is a numpy array
similarity_scores = np.array(similarities_df['TotalSimilarity_softmax'].tolist())

# Perform the dot product
result = np.dot(similarity_scores, data)

# Convert the result to a list
result_list = result.tolist() + [0]

item_list_path = r'E:\OneDrive - National University of Singapore\code\NUS-ISS\Project1\IRS-PM-2024-09-10-GRP8-JobRecommendationSystem\Recommendation_System\KGAT\datasets\ours\item_list.txt'

with open(item_list_path, 'r') as f:
    item_list = f.readlines()
    
item = [line.split() for line in item_list]
df = pd.DataFrame(item)
df['result_list'] = result_list
df.drop(columns=[0, 1], inplace=True)
df.rename(columns={2: 'job_id'}, inplace=True)
final_result = df.sort_values(by='result_list', ascending=False)

final_result

,job_id,result_list
7308,1578660035545093,0.015100
6303,1697415787967939,0.014879
826,2854243280141414,0.014873
2594,1768735516082674,0.014834
130,505789321323157,0.014748
...,...,...
151,1437485129918240,-0.019905
5920,3087145137798087,-0.019916
6000,1824055535842521,-0.019920
805,1043606419938718,-0.019972


In [37]:
job_sg_path = r'E:\OneDrive - National University of Singapore\code\NUS-ISS\Project1\IRS-PM-2024-09-10-GRP8-JobRecommendationSystem\Recommendation_System\New_User\job_clean.csv'

job_sg_df = pd.read_csv(job_sg_path)

job_sg_df = job_sg_df[['Job Id', 'Job Title', 'Job Description', 'Company', 'skills', 'location', 'Work Type', 'Salary Range']]

In [38]:
# Ensure 'job_id' in final_result is of the same type as 'Job Id' in job_sg_df
final_result['job_id'] = final_result['job_id'].astype(str)
job_sg_df['Job Id'] = job_sg_df['Job Id'].astype(str)

# Merge final_result with job_sg_df on job_id and Job Id
merged_result = final_result.merge(job_sg_df, left_on='job_id', right_on='Job Id', how='left')

# Drop the redundant 'Job Id' column
merged_result.drop(columns=['Job Id'], inplace=True)

# Display the merged result
merged_result

,job_id,result_list,Job Title,Job Description,Company,skills,location,Work Type,Salary Range
0,1578660035545093,0.015100,Marketing Coordinator,Event Marketing Coordinators focus on marketin...,Adani Ports and Special Economic Zone,Event planning and management Budgeting and fi...,Singapore,Temporary,$60K-$98K
1,1697415787967939,0.014879,Procurement Manager,"Analyze procurement data, identify cost-saving...",GAIL (India) Limited,Procurement processes Vendor assessment Contra...,Singapore,Part-Time,$58K-$117K
2,2854243280141414,0.014873,Human Resources Manager,Talent Acquisition Managers oversee the recrui...,Elevance Health,Talent sourcing Interviewing Onboarding Recrui...,Singapore,Contract,$57K-$104K
3,1768735516082674,0.014834,Human Resources Manager,Talent Acquisition Managers oversee the recrui...,International Flavors & Fragrances,Talent sourcing Interviewing Onboarding Recrui...,Singapore,Full-Time,$64K-$86K
4,505789321323157,0.014748,Marketing Coordinator,Marketing Specialists develop and implement ma...,Fifth Third Bancorp,Market research and analysis Content creation ...,Singapore,Temporary,$62K-$109K
...,...,...,...,...,...,...,...,...,...
7420,1437485129918240,-0.019905,Chemical Analyst,"In the role of a Quality Control Analyst, you ...",Carnival Corporation & plc,Quality control Quality assurance QC testing m...,Singapore,Contract,$63K-$108K
7421,3087145137798087,-0.019916,Procurement Manager,"Analyze procurement data, identify cost-saving...",Continental AG,Procurement processes Vendor assessment Contra...,Singapore,Full-Time,$61K-$90K
7422,1824055535842521,-0.019920,Chemical Analyst,"In the role of a Quality Control Analyst, you ...",Franklin Resources,Quality control Quality assurance QC testing m...,Singapore,Intern,$65K-$97K
7423,1043606419938718,-0.019972,Procurement Manager,Promote diversity and inclusion in the supply ...,Volkswagen Group,Supplier diversity programs Diversity and incl...,Singapore,Contract,$62K-$101K


In [39]:
Jobs = []
for idx, result in merged_result.iterrows():
    job = {}
    job['jobId'] = result['job_id']
    job['jobTitle'] = result['Job Title']
    job['jobDescription'] = result['Job Description']
    job['companyName'] = result['Company']
    job['skills'] = result['skills']
    job['location'] = result['location']
    job['employmentType'] = result['Work Type']
    job['salary'] = result['Salary Range']

Jobs.append(job)
Jobs

[{'jobId': '2762655424740902',
  'jobTitle': 'Web Designer',
  'jobDescription': 'Frontend Web Designers create the visual elements and user interfaces of websites. They use HTML, CSS, and JavaScript to design responsive, user-friendly web pages, ensuring a seamless and visually appealing online experience.',
  'companyName': 'ARKO',
  'skills': 'Frontend web development HTML/CSS Responsive design User interface (UI) design Adobe Creative Suite Collaboration skills',
  'location': 'Singapore',
  'employmentType': 'Part-Time',
  'salary': '$60K-$102K'},
 {'jobId': '1181577271565115',
  'jobTitle': 'IT Manager',
  'jobDescription': 'An Infrastructure Manager is responsible for the operation and maintenance of an organizations IT infrastructure, including servers, networks, and data centers.',
  'companyName': 'Rightmove',
  'skills': 'Network administration System administration Virtualization technologies (e.g., VMware, Hyper-V) Storage management IT security Disaster recovery planning 

In [41]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
from scipy.special import softmax

app = Flask(__name__)

@app.route('/recommend_jobs', methods=['POST'])
def recommend_jobs():
    usecols=['Respondent',        #id
    'FormalEducation',    #degree
    'UndergradMajor',     #major
    'LanguageWorkedWith', 'DatabaseWorkedWith', 'PlatformWorkedWith', 'IDE', 'OperatingSystem', #skills
    'YearsCoding',        #projectExperience
    'Hobby',              #hobby
    'DevType',            #desiredJob
    ]

    user_path = r'E:\OneDrive - National University of Singapore\code\NUS-ISS\Project1\IRS-PM-2024-09-10-GRP8-JobRecommendationSystem\Recommendation_System\New_User\user_clean.csv'

    df = pd.read_csv(user_path, usecols=usecols)
    df = df.rename(columns={
    'Respondent': 'id',
    'FormalEducation': 'degree',
    'UndergradMajor': 'major',
    'LanguageWorkedWith': 'skills_1',
    'DatabaseWorkedWith': 'skills_2',
    'PlatformWorkedWith': 'skills_3',
    'IDE': 'skills_4',
    'OperatingSystem': 'skills_5',
    'YearsCoding': 'projectExperience',
    'Hobby': 'hobby',
    'DevType': 'desiredJob',
    })

    df['skills_1'] = df['skills_1'].str.split(';')
    df['skills_2'] = df['skills_2'].str.split(';')
    df['skills_3'] = df['skills_3'].str.split(';')
    df['skills_4'] = df['skills_4'].str.split(';')
    df['skills_5'] = df['skills_5'].str.split(';')

    df['skills'] = df[['skills_1', 'skills_2', 'skills_3', 'skills_4', 'skills_5']].apply(lambda x: sum([i for i in x if isinstance(i, list)], []), axis=1)
    df.drop(columns=['skills_1', 'skills_2', 'skills_3', 'skills_4', 'skills_5'], inplace=True)

    df['hobby'] = df['hobby'].apply(lambda x: 'True' if x == 'Yes' else 'False')
    df['desiredJob'] = df['desiredJob'].str.split(';')
    df['desiredJob'] = df[['desiredJob']].apply(lambda x: sum([i for i in x if isinstance(i, list)], []), axis=1)

    new_data = request.get_json()
    print(new_data)

    df = df._append(new_data, ignore_index=True)
    df_onehot = pd.get_dummies(df['degree'])
    df['degree'] = df_onehot.apply(lambda x: x.values.tolist(), axis=1)
    df_onehot = pd.get_dummies(df['major'])
    df['major'] = df_onehot.apply(lambda x: x.values.tolist(), axis=1)
    df_onehot = pd.get_dummies(df['projectExperience'])
    df['projectExperience'] = df_onehot.apply(lambda x: x.values.tolist(), axis=1)
    # Function to calculate Jaccard similarity for binary vectors (one-hot encoded)
    def jaccard_similarity_binary(vector1, vector2):
        set1, set2 = set(np.where(vector1 == 1)[0]), set(np.where(vector2 == 1)[0])
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        if union == 0:
            return 0
        return intersection / union

    # Function to calculate Jaccard similarity for lists of strings
    def jaccard_similarity(list1, list2):
        set1, set2 = set(map(str.lower, list1)), set(map(str.lower, list2))
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        if union == 0:
            return 0
        return intersection / union


    # Convert string representations of lists back to actual lists
    list_columns = ['degree', 'major', 'desiredJob', 'projectExperience', 'skills']
    for col in list_columns:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)

    # Separate the new user (last row) from the rest of the data
    new_user = df.iloc[-1]
    other_users = df[:-1]

    # Similarity calculation for each feature

    # 1. Similarity for 'hobby' (boolean comparison)
    hobby_similarity = other_users['hobby'].apply(lambda x: 100 if x == new_user['hobby'] else 0)

    # 2. Similarity for one-hot encoded features (degree, major, projectExperience)
    onehot_similarity = []
    for idx, other_user in other_users.iterrows():
        degree_sim = jaccard_similarity_binary(np.array(new_user['degree']), np.array(other_user['degree']))
        major_sim = jaccard_similarity_binary(np.array(new_user['major']), np.array(other_user['major']))
        project_sim = jaccard_similarity_binary(np.array(new_user['projectExperience']), np.array(other_user['projectExperience']))
        onehot_similarity.append((degree_sim + major_sim + project_sim) / 0.03)

    # 3. Similarity for lists of strings (desiredJob, skills)
    list_similarity = []
    for idx, other_user in other_users.iterrows():
        desiredJob_sim = jaccard_similarity(new_user['desiredJob'], other_user['desiredJob'])
        skills_sim = jaccard_similarity(new_user['skills'], other_user['skills'])
        list_similarity.append((desiredJob_sim + skills_sim) / 0.02)

    # Combine similarities with equal weights
    total_similarities = hobby_similarity * 0.05 + np.array(onehot_similarity) * 0.45 + np.array(list_similarity) *0.5

    # Create a DataFrame to display similarities
    similarities_df = pd.DataFrame({
        'UserIndex': other_users.id,
        'HobbySimilarity': hobby_similarity,
        'OneHotSimilarity': onehot_similarity,
        'ListSimilarity': list_similarity,
        'TotalSimilarity': total_similarities
    })
    print(similarities_df)

    # Sort by total similarity
    # similarities_df = similarities_df.sort_values(by='UserIndex', ascending=True)

    # Display the top results
    # Apply softmax to the 'TotalSimilarity' column
    similarities_df['TotalSimilarity_softmax'] = softmax(similarities_df['TotalSimilarity'])
    data = np.load(r'E:\OneDrive - National University of Singapore\code\NUS-ISS\Project1\IRS-PM-2024-09-10-GRP8-JobRecommendationSystem\Recommendation_System\KGAT\trained_model\KGAT\ours\pretrain0\cf_scores.npy')
    # Replace -inf values with 0
    data[data == -np.inf] = np.min(data[data != -np.inf])

    # Normalize each row
    data = data / np.linalg.norm(data, axis=1, keepdims=True)
    # Ensure that similarities_df['TotalSimilarity_softmax'] is a numpy array
    similarity_scores = np.array(similarities_df['TotalSimilarity_softmax'].tolist())

    # Perform the dot product
    result = np.dot(similarity_scores, data)

    # Convert the result to a list
    result_list = result.tolist() + [0]

    item_list_path = r'E:\OneDrive - National University of Singapore\code\NUS-ISS\Project1\IRS-PM-2024-09-10-GRP8-JobRecommendationSystem\Recommendation_System\KGAT\datasets\ours\item_list.txt'

    with open(item_list_path, 'r') as f:
        item_list = f.readlines()
        
    item = [line.split() for line in item_list]
    df = pd.DataFrame(item)
    df['result_list'] = result_list
    df.drop(columns=[0, 1], inplace=True)
    df.rename(columns={2: 'job_id'}, inplace=True)
    final_result = df.sort_values(by='result_list', ascending=False)
    job_sg_path = r'E:\OneDrive - National University of Singapore\code\NUS-ISS\Project1\IRS-PM-2024-09-10-GRP8-JobRecommendationSystem\Recommendation_System\New_User\job_clean.csv'
    job_sg_df = pd.read_csv(job_sg_path)
    job_sg_df = job_sg_df[['Job Id', 'Job Title', 'Job Description', 'Company', 'skills', 'location', 'Work Type', 'Salary Range']]
    # Ensure 'job_id' in final_result is of the same type as 'Job Id' in job_sg_df
    final_result['job_id'] = final_result['job_id'].astype(str)
    job_sg_df['Job Id'] = job_sg_df['Job Id'].astype(str)

    # Merge final_result with job_sg_df on job_id and Job Id
    merged_result = final_result.merge(job_sg_df, left_on='job_id', right_on='Job Id', how='left')

    # Drop the redundant 'Job Id' column
    merged_result.drop(columns=['Job Id'], inplace=True)

    Jobs = []
    for idx, result in merged_result.iterrows():
        job = {}
        job['jobId'] = result['job_id']
        job['jobTitle'] = result['Job Title']
        job['jobDescription'] = result['Job Description']
        job['companyName'] = result['Company']
        job['skills'] = result['skills']
        job['location'] = result['location']
        job['employmentType'] = result['Work Type']
        job['salary'] = result['Salary Range']
        Jobs.append(job)
    print(Jobs[:20])
    return jsonify(Jobs[:20])

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.146.142:8080
Press CTRL+C to quit


{'userId': 1, 'passwd': None, 'userName': None, 'phoneNumber': None, 'emailAddress': None, 'hometown': None, 'degree': 'Master’s degree (MA, MS, M.Eng., MBA, etc.)', 'major': 'Computer science, computer engineering, or software engineering', 'skills': ['c', ' c++', ' python', ' java', ' javascript'], 'projectExperience': '0-2 years', 'hobby': 'True', 'desiredJob': ['']}
      UserIndex  HobbySimilarity  OneHotSimilarity  ListSimilarity  \
0          96.0              100         33.333333             0.0   
1         105.0              100         33.333333             0.0   
2         122.0              100         33.333333             0.0   
3         294.0              100          0.000000             0.0   
4         297.0              100         33.333333             0.0   
...         ...              ...               ...             ...   
1381    62517.0              100         33.333333             0.0   
1382    65734.0                0         33.333333             0.0 

192.168.146.170 - - [26/Oct/2024 16:42:47] "POST /recommend_jobs HTTP/1.1" 200 -


[{'jobId': '2629102261205843', 'jobTitle': 'Web Developer', 'jobDescription': 'Backend Web Developers build and maintain the server-side logic and infrastructure required for web applications. They work on data storage, APIs, and server communication to support the functionality of web applications.', 'companyName': 'Exxon Mobil Corporation', 'skills': 'Server-side programming (e.g., Node.js, Python) Database management (e.g., SQL, MongoDB)', 'location': 'Singapore', 'employmentType': 'Contract', 'salary': '$62K-$130K'}, {'jobId': '2674018506735932', 'jobTitle': 'Database Developer', 'jobDescription': 'NoSQL Database Engineers work with non-relational databases to store and manage large volumes of unstructured or semi-structured data. They design data models, optimize database performance, and ensure scalability and data consistency.', 'companyName': 'Ovintiv', 'skills': 'NoSQL databases (e.g., MongoDB, Cassandra) Data modeling Data migration Performance tuning Scalability Distributed 

...

[{'jobId': '1832388118151083', 'jobTitle': 'Data Analyst', 'jobDescription': 'Data Quality Analysts ensure the accuracy, completeness, and consistency of data within an organization. They develop and implement data quality standards, perform data validation, and identify and resolve data quality issues to maintain data integrity.', 'companyName': 'BYD Company', 'skills': 'Data quality assessment and improvement Data profiling and validation Knowledge of data quality tools (e.g., Informatica, Talend) Data governance principles SQL for data analysis Attention to detail and data accuracy', 'location': 'Singapore', 'employmentType': 'Temporary', 'salary': '$60K-$119K'}, {'jobId': '1463485379657957', 'jobTitle': 'Software Engineer', 'jobDescription': 'A Backend Developer is tasked with building and maintaining the server-side logic and database infrastructure of web applications. They work on the core functionality, data storage, and server communication to ensure the application runs smoot

192.168.146.170 - - [26/Oct/2024 17:02:08] "POST /recommend_jobs HTTP/1.1" 200 -


[{'jobId': '1899722720713087', 'jobTitle': 'Front-End Developer', 'jobDescription': 'An Accessibility Developer focuses on making digital products and websites accessible to individuals with disabilities, ensuring compliance with accessibility standards.', 'companyName': 'Deutsche Annington Immobilien SE (Now Vonovia SE)', 'skills': 'Web accessibility standards (e.g., WCAG) Assistive technologies Accessibility testing tools HTML and CSS for accessibility ARIA (Accessible Rich Internet Applications) User experience (UX) for accessibility Usability testing with individuals with disabilities Collaboration Attention to detail', 'location': 'Singapore', 'employmentType': 'Intern', 'salary': '$59K-$98K'}, {'jobId': '584678341069526', 'jobTitle': 'Web Designer', 'jobDescription': 'E-commerce Web Designers specialize in designing and optimizing online stores and shopping websites. They focus on user experience, product presentation, and conversion rate optimization to drive online sales and cu

192.168.146.170 - - [26/Oct/2024 17:02:56] "POST /recommend_jobs HTTP/1.1" 200 -


[{'jobId': '1727189231682740', 'jobTitle': 'IT Administrator', 'jobDescription': 'System Administrators manage and maintain computer systems and networks. They handle software installations, security measures, and troubleshooting to ensure system reliability.', 'companyName': 'Boise Cascade', 'skills': 'System administration Network administration Troubleshooting Security management Server maintenance Backup and recovery', 'location': 'Singapore', 'employmentType': 'Intern', 'salary': '$65K-$107K'}, {'jobId': '1181577271565115', 'jobTitle': 'IT Manager', 'jobDescription': 'An Infrastructure Manager is responsible for the operation and maintenance of an organizations IT infrastructure, including servers, networks, and data centers.', 'companyName': 'Rightmove', 'skills': 'Network administration System administration Virtualization technologies (e.g., VMware, Hyper-V) Storage management IT security Disaster recovery planning Capacity planning Monitoring and troubleshooting Cloud infrastr